# Exchange FYTD

In [4]:
import pandas as pd
import pandas_datareader.data as pd_dr_web
from datetime import datetime
from typing import Union, List

In [20]:
def add_usd_conversion(df_raw: pd.DataFrame, cols_to_exclude: Union[List[str], None] = None) -> pd.DataFrame:

    start_date, end_date = df_raw['date'].min(), df_raw['date'].max()

    # Exclude portfolio columns
    df_filtered = df_raw.query("portfolio not in @cols_to_exclude").copy() if cols_to_exclude else df_raw.copy()

    # df_payments_ytd.reset_index(drop=True, inplace=True)

    df_conversion = pd_dr_web.DataReader(
        ["DEXUSUK", "DEXCAUS", "DEXUSAL", "DEXUSEU", "DEXSIUS", "DEXSZUS"], 
        "fred",
        start_date,
        end_date
    )

    # First, create a complete date range
    date_range = pd.date_range(
        start=start_date,
        end=end_date,
        freq='D'
    )

    # Reindex the dataframe with the complete date range and back fill the values
    df_conversion = df_conversion.reindex(date_range).bfill()

    # Create a dictionary for each currency's conversion rates
    conversion_rates = {
        'USD': 1.0,
        'GBP': df_conversion['DEXUSUK'],
        'CAD': 1/df_conversion['DEXCAUS'],  # Inverting for division
        'AUD': df_conversion['DEXUSAL'],
        'EUR': df_conversion['DEXUSEU'],
        'SGD': 1/df_conversion['DEXSIUS'],  # Inverting for division
        'CHF': 1/df_conversion['DEXSZUS']   # Inverting for division
    }

    # Create the conversion multiplier series based on currency
    df_filtered['multiplier'] = df_filtered.apply(
        lambda row: 1 if row["currency"] == "USD"
            else conversion_rates[row["currency"]][row["date"]],
        axis=1
    )

    # Multiply amount by the conversion rate
    df_filtered['amount_usd'] = (
        df_filtered['amount'] * df_filtered['multiplier']
    ).round(2)

    return df_filtered

In [21]:
COLS_TO_EXCLUDE = ['One for the World Discretionary Fund', 'One for the World Operating Costs']

In [22]:
df_payments_converted_to_usd = add_usd_conversion(df_raw=pd.read_json('payments.json'), cols_to_exclude=COLS_TO_EXCLUDE)

## Add `year` column

In [27]:
df_payments_converted_to_usd['year'] = df_payments_converted_to_usd.date.dt.year

In [ ]:
df_payments_converted_to_usd.to_csv('payments_amount_usd.csv')

: 